# Approach 2.5 / 3.5

Each of the approaches tested so far, despite being used previously in the literature (at least to the extent that I understand them) have failed to out perform just randomly selecting samples. I suspect this could be due to one and/or another issue with the current acquisition strategy. Each of these approaches use some method to find the variance of model predictions to find the most uncertain sequences for the next batch. However, those uncertain sequences might somehow correspond to noisy measurements (measurements with lower read counts, especially if they tend to be the sequences with more mutations), and they might not be sufficiently diverse.

Given these hypotheses for what's going wrong, I may be able to solve them by ensuring the next sample is uncertain as well as diverse. I'd be interested in testing when its just diverse and when its uncertain and diverse compared to the random baseline.

But how do we ensure diversity? Well, since we are conveniently using a language model, we should be able to just use the language model embeddings. So, all I need to do is figure out how to get the embeddings, measure their distance, and integrate some amount of distance optimization into my acquistion function.

Note: Below, I'm going to extract the embeddings from the last hidden state of the CLS token, since that's the standard model I've been working with setting up the structure of active learning. However, other fine-tuning approaches should make use of different embeddings. For example, if you're using only the mutant residues as the final state input to your output layer, you should probably use those embeddings for diversity. Another decision point is whether to use the embeddings of your fine-tuned model or a fresh ESM2 model. I'm going to work with just the fine-tuned model here.

Here's the algorithm I'll employ:
1. Train ensemble on random initial set.
2. Get variances from model predictions.
3. Filter samples with top k% of variances.
4. Retrieve embeddings for those samples.
5. For each sample in the unlabeled pool, calculate its distance from each of the samples in the current batch.
6. Find the minimum distance from those for each candidate.
7. Add candidate c whose minimum distance from the current batch is the greatest and remove it from the unlabeled pool.
8. Calculate the distance of just-added candidate c from each of the remaining unlabeled pool. Add these distances to the distances list for each sample.
9. Repeat 6-8 until n samples are added to the labeled samples.

Actually, now that I spell out the algorithm, I'm realizing that I should also test just a random choice from the top k% of variances to make sure that the embeddings even add anything.

In [1]:
from scripts.data_utils import train_val_test_split
from scripts.config import (
    DATA_PATH, 
    SEQUENCE_COL, 
    SCORE_COL, 
    TOK_MODEL, 
    VAL_SPLIT,
    TEST_SPLIT,
    BATCH_SIZE,
    RANDOM_SEED,
)

training_pool, val_dataloader, test_dataloader = train_val_test_split(
    DATA_PATH,
    SEQUENCE_COL,
    SCORE_COL,
    TOK_MODEL,
    VAL_SPLIT,
    TEST_SPLIT,
    BATCH_SIZE,
    RANDOM_SEED
)

In [2]:
import torch
from torch.utils.data import Subset, DataLoader
import numpy as np

from scripts.training import initialize_and_train_new_model
from scripts.acquisition import get_pool_predictions

def get_bootstrap_sample(labeled_indices, pool_dataset, train_dataloader_batch_size):
    bootstrap_indices = np.random.choice(labeled_indices, size=int(0.9*len(labeled_indices)),replace=True)
    bootstrap_subset = Subset(pool_dataset, bootstrap_indices)
    bootstrap_dataloader = DataLoader(bootstrap_subset, batch_size=train_dataloader_batch_size, shuffle=True)
    return bootstrap_dataloader

def train_ensemble(
        n_models, 
        model_name, 
        approach,
        learning_rate,
        weight_decay,
        epochs,
        labeled_indices,
        train_dataloader_batch_size,
        pool_dataset, 
        pool_dataloader, 
        val_dataloader,
        patience
        ):
    
    # define list to store predictions as each model is trained then evaluated
    ensemble_predictions = []
    
    for i in range(n_models):
        print(f"\nTraining Model {i+1}...")
        # set a changing manual seed
        torch.manual_seed(i)
        torch.cuda.manual_seed(i)

        # get bootstrap sample from labeled dataset
        bootstrap_dataloader = get_bootstrap_sample(labeled_indices, pool_dataset, train_dataloader_batch_size)

        # initialize and train a new model
        model = initialize_and_train_new_model(approach, model_name, learning_rate, weight_decay, epochs, bootstrap_dataloader, val_dataloader, patience)
        
        # get model predictions on pool dataloader, append to ensemble predictions list
        pool_preds = get_pool_predictions(model, pool_dataloader)
        ensemble_predictions.append(pool_preds)

    # stack ensemble predictions to create tensor of shape (n_models, n_unlabeled_samples)
    ensemble_predictions = torch.stack(ensemble_predictions, dim=0)
    print("Ensemble training complete, submitting predictions for next cycle.")
    # return list of ensemble predictions
    return ensemble_predictions

### Random choice from top % variances  

In [9]:
import numpy as np
from torch.utils.data import Subset, DataLoader

# acquire new batch, randomly if no scores given, top "batch_size_to_acquire" if given
def acquire_new_batch(
    dataset, 
    train_dataloader_batch_size, 
    pool_dataloader_batch_size, 
    initial_batch_size,
    top_score_fraction, 
    batch_size_to_acquire, 
    labeled_indices, 
    unlabeled_indices, 
    acquisition_scores=None
    ):

    # if initial batch, when there are no acquisition scores, select randomly
    if acquisition_scores is None:
        initial_batch_size = min(initial_batch_size, len(unlabeled_indices))
        indices_to_acquire = np.random.choice(unlabeled_indices, size=initial_batch_size, replace=False)
    
    # else select based on top acquisition scores
    else:
        # make sure we don't overshoot samples to acquire if on the final batch
        batch_size_to_acquire = min(batch_size_to_acquire, len(acquisition_scores))
        # detmine the number of top scorers to select from
        num_top_scorers = int(top_score_fraction * len(unlabeled_indices))
        # get the indicies of the top acquisition scores (num of samples)
        top_indices = acquisition_scores.topk(num_top_scorers).indices
        # choose a random set of indices from these top scorers
        top_k_indices = np.random.choice(top_indices.cpu().numpy(), size=batch_size_to_acquire, replace=False)
        # use these to find the indices that map back to the original dataset
        indices_to_acquire = unlabeled_indices[top_k_indices]
    
    # update the indices lists
    labeled_indices = np.concatenate([labeled_indices, indices_to_acquire])
    unlabeled_indices = np.setdiff1d(unlabeled_indices, indices_to_acquire, assume_unique=True)
    
    # create new subsets and dataloaders
    train_subset = Subset(dataset, labeled_indices.tolist())
    pool_subset = Subset(dataset, unlabeled_indices.tolist())
    train_dataloader = DataLoader(train_subset, batch_size=train_dataloader_batch_size, shuffle=True)
    pool_dataloader = DataLoader(pool_subset, batch_size=pool_dataloader_batch_size, shuffle=False)
    
    return train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices

In [10]:
from pathlib import Path
import pandas as pd

from scripts.acquisition import get_variances
from scripts.training import initialize_and_train_new_model, test_model
from scripts.campaigns import run_standard_finetuning


def get_learning_curves(
        n_samples,
        initial_n_samples,
        top_score_fraction,
        n_samples_per_batch,
        model_name, 
        approach,
        learning_rate, 
        weight_decay, 
        epochs, 
        training_pool, 
        train_dataloader_batch_size,
        pool_dataloader_batch_size,
        val_dataloader, 
        test_dataloader,
        patience=5,
        n_models=5,
        results_path="active_vs_standard_learning_curves.csv"
):
    results_path = Path(results_path)
    results_dir = results_path.parent
    results_dir.mkdir(parents=True, exist_ok=True)

    # Load existing results if the file exists, otherwise start with a fresh DataFrame.
    if results_path.exists():
        all_results_df = pd.read_csv(results_path)
    else:
        all_results_df = pd.DataFrame()
    
    total_pool_size = len(training_pool)
    unlabeled_indices = np.arange(total_pool_size)
    labeled_indices = np.array([], dtype=np.int64)

    ensemble_predictions = None
    current_cycle = 1
    total_cycles = int(np.ceil((n_samples-initial_n_samples)/n_samples_per_batch)) + 1
    
    while len(labeled_indices) < n_samples and len(unlabeled_indices) > 0:
        print(f"\nCycle {current_cycle}/{total_cycles}\n-------------------------------------------------")

        # on the first cycle, choose random samples of initial_n_samples size
        if ensemble_predictions is None:
            print(f"Choosing initial {initial_n_samples} samples randomly...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, train_dataloader_batch_size, pool_dataloader_batch_size, initial_n_samples, top_score_fraction, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=None
            )
        # each other time, use the n_samples_per_batch with acquisition scores to select
        else:
            scores = get_variances(ensemble_predictions, f"{results_dir}/variances{current_cycle}.csv")
            print(f"Selecting new data points...")
            train_dataloader, pool_dataloader, labeled_indices, unlabeled_indices = acquire_new_batch(
                training_pool, train_dataloader_batch_size, pool_dataloader_batch_size, initial_n_samples, top_score_fraction, n_samples_per_batch, labeled_indices, unlabeled_indices, acquisition_scores=scores
            )
        
        # give message when loop ends
        if len(unlabeled_indices) == 0:
            print("Unlabeled pool is empty. Proceeding to final model training.")
            break
        
        # evaluate active vs standard
        final_results = []

        # active
        print(f"\nTraining and evaluating model using {len(labeled_indices)} actively selected samples...")
        model_active = initialize_and_train_new_model(approach, model_name, learning_rate, weight_decay, epochs, train_dataloader, val_dataloader, patience, return_history=False)
        results_active = test_model(model_active, test_dataloader, return_results=True)
        results_active = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'active',
            **results_active
        }
        final_results.append(results_active)

        # standard
        print(f"\nTraining and evaluating model using {len(labeled_indices)} randomly selected samples...")
        model_standard, _ = run_standard_finetuning(len(labeled_indices), approach, model_name, train_dataloader_batch_size, learning_rate, weight_decay, epochs, training_pool, val_dataloader, patience)
        results_standard = test_model(model_standard, test_dataloader, return_results=True)
        results_standard = {
            'changing_var': 'n_samples',
            'local_exp_idx': current_cycle-1,
            'value': len(labeled_indices),
            'training_method': 'standard',
            **results_standard
        }
        final_results.append(results_standard)
        # save to disk each time to save progress
        results_df = pd.DataFrame(final_results)
        all_results_df = pd.concat([all_results_df, results_df], ignore_index=True)
        all_results_df.to_csv(results_path, index=False)
        print(f"Progress for experiment {current_cycle-1} appended to {results_path}")

        # if it's the last cycle, skip ensemble predictions
        if (current_cycle == total_cycles):
            print("Experiments complete.")
            break

        print("Starting ensemble training and pool evaluation...")
        ensemble_predictions = train_ensemble(n_models, model_name, approach, learning_rate, weight_decay, epochs, labeled_indices, train_dataloader_batch_size, training_pool, pool_dataloader, val_dataloader, patience)
    
        current_cycle += 1
    return all_results_df

In [ ]:
from scripts.config import (
    MODEL_NAME,
    APPROACH,
    LEARNING_RATE,
    WEIGHT_DECAY,
    EPOCHS,
    POOL_BATCH_SIZE,
    PATIENCE,
    N_MODELS,
    TOP_SCORE_FRACTION,
)

get_learning_curves(
    n_samples=256,
    initial_n_samples=16,
    top_score_fraction=TOP_SCORE_FRACTION,
    n_samples_per_batch=16,
    model_name=MODEL_NAME,
    approach=APPROACH,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    epochs=EPOCHS,
    training_pool=training_pool,
    train_dataloader_batch_size=BATCH_SIZE,
    pool_dataloader_batch_size=POOL_BATCH_SIZE,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    patience=PATIENCE,
    n_models=N_MODELS,
    results_path='results/04_adding_diversity/random_from_top_fraction/active_vs_standard_learning_curve.csv'
)


Cycle 1/16
-------------------------------------------------
Choosing initial 16 samples randomly...

Training and evaluating model using 16 actively selected samples...


/home/oscar/miniconda3/envs/plm-active-learning/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
[Training]: 100%|██████████| 50/50 [00:35<00:00,  1.42it/s]


Train Loss: 0.0336 | Val Loss: 0.1788 | SpearmanR: 0.4060


[Testing]: 100%|██████████| 25/25 [00:00<00:00, 127.40it/s]



Training and evaluating model using 16 randomly selected samples...


...

[Training]:  82%|████████▏ | 41/50 [00:23<00:05,  1.74it/s]


Early stopping triggered after 10 epochs with no improvement.
Train Loss: 0.0144 | Val Loss: 0.1526 | SpearmanR: 0.5292


[Surveying]: 100%|██████████| 24/24 [00:01<00:00, 17.32it/s]


Ensemble training complete, submitting predictions for next cycle.

Cycle 14/16
-------------------------------------------------
Saving variance distribution to results/04_adding_diversity/random_from_top_fraction/variances14.csv...
Save complete.
Selecting new data points...

Training and evaluating model using 224 actively selected samples...


[Training]:  88%|████████▊ | 44/50 [00:39<00:03,  1.96it/s]

In [ ]:
lc_df = pd.read_csv("results/04_adding_diversity/random_from_top_fraction/active_vs_standard_learning_curve.csv")
lc_df.head()

changing_var  local_exp_idx  value training_method  avg_test_loss  \
0    n_samples              0     16          active       0.197894   
1    n_samples              0     16        standard       0.320916   
2    n_samples              1     32          active       0.184044   
3    n_samples              1     32        standard       0.173450   
4    n_samples              2     48          active       0.236712   

   spearmanr  pearsonr  final_mse  
0   0.382034  0.373106   0.198538  
1   0.270746  0.294286   0.320806  
2   0.545784  0.502709   0.184418  
3   0.416550  0.422225   0.173986  
4   0.378425  0.410424   0.237579

In [ ]:
import plotly.express as px

fig = px.line(lc_df, 'value', 'spearmanr',color="training_method")

fig.update_layout(
    xaxis_title="Number of Training Samples",
    yaxis_title="Spearman Correlation Coefficient"
)

fig.show()

In [ ]:
import pandas as pd
import glob
from pathlib import Path

file_pattern = "results/04_adding_diversity/random_from_top_fraction/variances*.csv"
variance_files = glob.glob(file_pattern)
variance_files.sort()

all_variances_dfs = []

for filepath in variance_files:
    temp_df = pd.read_csv(filepath)
    column_name = Path(filepath).stem
    temp_df = temp_df.rename(columns={'variance': column_name})
    all_variances_dfs.append(temp_df)

final_df = pd.concat(all_variances_dfs, axis=1)

final_df

variances10  variances11  variances12  variances13  variances14  \
0        0.149963     0.011875     0.025784     0.021565     0.042797   
1        0.050087     0.005050     0.020347     0.031385     0.019446   
2        0.069689     0.014829     0.010998     0.004873     0.013083   
3        0.030903     0.022928     0.014478     0.082185     0.103137   
4        0.036251     0.021226     0.081336     0.022297     0.035207   
...           ...          ...          ...          ...          ...   
3147          NaN          NaN          NaN          NaN          NaN   
3148          NaN          NaN          NaN          NaN          NaN   
3149          NaN          NaN          NaN          NaN          NaN   
3150          NaN          NaN          NaN          NaN          NaN   
3151          NaN          NaN          NaN          NaN          NaN   

      variances15  variances16  variances2  variances3  variances4  \
0        0.023504     0.020772    0.013999    0.028303    0.051192   
1        0.037039     0.030839    0.119323    0.064065    0.027289   
2        0.016777     0.035694    0.028558    0.052594    0.060716   
3        0.105877     0.029240    0.044814    0.054224    0.040302   
4        0.022637     0.013253    0.083888    0.057288    0.017450   
...           ...          ...         ...         ...         ...   
3147          NaN          NaN    0.032242         NaN         NaN   
3148          NaN          NaN    0.089120         NaN         NaN   
3149          NaN          NaN    0.029723         NaN         NaN   
3150          NaN          NaN    0.050000         NaN         NaN   
3151          NaN          NaN    0.041384         NaN         NaN   

      variances5  variances6  variances7  variances8  variances9  
0       0.048048    0.027420    0.055021    0.014279    0.049370  
1       0.024033    0.024117    0.022003    0.027090    0.043563  
2       0.149823    0.034743    0.123635    0.090861    0.006818  
3       0.059472    0.037716    0.021208    0.004541    0.026667  
4       0.041651    0.015074    0.017717    0.039450    0.071926  
...          ...         ...         ...         ...         ...  
3147         NaN         NaN         NaN         NaN         NaN  
3148         NaN         NaN         NaN         NaN         NaN  
3149         NaN         NaN         NaN         NaN         NaN  
3150         NaN         NaN         NaN         NaN         NaN  
3151         NaN         NaN         NaN         NaN         NaN  

[3152 rows x 15 columns]

In [ ]:
columns_to_plot = ['variances2', 'variances9', 'variances15']

# "Melt" the selected columns into a long-format DataFrame
melted_df = final_df[columns_to_plot].melt(
    var_name='Cycle',      # New column for the original column names
    value_name='Variance'  # New column for the variance values
)

fig = px.histogram(
    data_frame=melted_df,
    x='Variance',                                  
    color='Cycle',                                 
    barmode='overlay',                             
    opacity=0.65,                                  
    histnorm='probability density',                
    title='Distribution of Variances Across Active Learning Cycles'
)

fig.show()

### With embeddings

In [ ]:
def get_sequence_embedding(model, sequence):
    return embedding

def get_all_embeddings(unlabeled_indices):
    return embeddings

def get_distance(embed_1, embed_2):
    return distance

def get_all_distances(unlabeled_embeddings):
    return all_distances

def get_next_sample():
    return

def acquire_new_batch():
    return